In [8]:
import wave
import matplotlib.pyplot as plt
from os import listdir
from os.path import join, isfile, basename, abspath, dirname
from PIL import Image
import numpy as np
from scipy.io import wavfile
from ipy_progressbar import ProgressBar

In [9]:
def get_wav_info(wav_file):
    frame_rate, sound_info = wavfile.read(wav_file)
    sound_info = np.trim_zeros(sound_info)
    return sound_info, frame_rate

In [23]:
def graph_spectrogram(wav_file, output_dir, cmap):
    file_path = abspath(wav_file)
    file_name = basename(file_path)
    sound_info, frame_rate = get_wav_info(wav_file)
    fig = plt.figure(num=None, figsize=(5, 5))
    plt.title("Spectrogram")
    spec = plt.specgram(sound_info, Fs=frame_rate, cmap=cmap)
    plt.xlim([0, len(sound_info)/frame_rate])
    plt.ylim([0,frame_rate/2])
    plt.axis("off")
    plt.savefig('spectrogram.png', bbox_inches=0)
    plt.close("all")
    with open('spectrogram.png', 'r+b') as f:
        with Image.open(f) as image:
            w, h = image.size
            image.crop((63, 50, w-50, h-63)).save(join(output_dir,file_name[:-4]+".png"), "png")

In [11]:
def graph_spectrogram_chunks(wav_file, chunksize_ms, NFFT, noverlap):
    file_path = abspath(wav_file)
    file_name = basename(file_path)
    output_dir = join(dirname(dirname(file_path)), "viridis_chunks")
    sound_info, frame_rate = get_wav_info(wav_file)
    chunksize = chunksize_ms/1000*frame_rate
    
    chunks = [sound_info[n*chunksize:min((n+1)*chunksize, len(sound_info))] for n in range(int(len(sound_info)/chunksize))]
    idx = 0
    for chunk in ProgressBar(chunks):
        #if not isfile(join(output_dir,file_name[:-4]+"_%04d" % (idx,)+".png")):
        fig = plt.figure(num=None, figsize=(5, 5))
        spec = plt.specgram(chunk, NFFT=NFFT, noverlap=noverlap, Fs=frame_rate, cmap="viridis")
        plt.xlim([0, len(chunk)/frame_rate])
        plt.axis("off")
        plt.savefig('spectrogram.png', bbox_inches='tight')
        plt.close("all")
        with open('spectrogram.png', 'r+b') as f:
            with Image.open(f) as image:
                w, h = image.size
                image.crop((51, 17, w-33, h-30)).save(join(output_dir,file_name[:-4]+"_%04d" % (idx,)+".png"), "png")
        idx += 1

Just creating filelists for the different IS tasks.

In [12]:
cold = '/home/maurice/Desktop/INTERSPEECH/Cold_dist/wav'
addressee = '/home/maurice/Desktop/INTERSPEECH/Addressee_dist/wav'
snore = '/home/maurice/Desktop/INTERSPEECH/Snore_dist/wav'
cold_files = [join(cold, f) for f in listdir(cold) if isfile(join(cold, f))]
addressee_files = [join(addressee, f) for f in listdir(addressee) if isfile(join(addressee, f))]
snore_files = [join(snore, f) for f in listdir(snore) if isfile(join(snore, f))]

In [25]:
files = snore_files #+ snore_files + snore_files
number_of_files = len(files)
output_dir = '/home/maurice/Desktop/INTERSPEECH/Snore_dist/viridis_test'
cmap = 'viridis'
for file in ProgressBar(files):
    graph_spectrogram(file, output_dir, cmap)

KeyboardInterrupt: 